# Tunix RT - Kaggle Submission Notebook

This notebook provides a single-session workflow for the Tunix Hack competition.

**Workflow:**
1. Install dependencies
2. Build/load dataset
3. Train model (JAX/Flax)
4. Generate predictions
5. Evaluate and score

**Runtime:** Kaggle TPU or GPU (recommended)
**Time:** ~1-2 hours for 100 steps


## 1. Setup


In [ ]:
# Install dependencies
!pip install -q jax[cuda12] flax optax orbax-checkpoint transformers datasets

print("✅ Setup complete")


## 2. Configuration


In [ ]:
# Training configuration
MODEL_NAME = "google/gemma-2-2b"
DATASET = "golden-v2"
MAX_STEPS = 100
OUTPUT_DIR = "./output/kaggle_run"
SEED = 42
DEVICE = "auto"  # auto-detect GPU/TPU

print(f"Model: {MODEL_NAME}")
print(f"Dataset: {DATASET}")
print(f"Max Steps: {MAX_STEPS}")
print(f"Output: {OUTPUT_DIR}")


## 3. Build Dataset


In [ ]:
# Build golden-v2 dataset (100 traces)
!python backend/tools/seed_golden_v2.py

# Verify dataset exists
!ls -l backend/datasets/golden-v2/

print("✅ Dataset ready")


## 4. Train Model


In [ ]:
# Train with JAX/Flax
!python training/train_jax.py --dataset {DATASET} --model_name {MODEL_NAME} --max_steps {MAX_STEPS} --device {DEVICE} --output_dir {OUTPUT_DIR} --seed {SEED}

print("✅ Training complete")


## 5. Results

Training complete! Check the output directory for:
- `checkpoint_*`: Model weights
- `metrics.jsonl`: Training metrics
- See docs/kaggle_submission.md for evaluation steps
